In [65]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [66]:
CLIENT_ID = '4IWPTPZBLHK1ISW0CQEP0IBTOX3W4OSIRB1DHH3VWM3PYXWU' # your Foursquare ID
CLIENT_SECRET = 'QTGRBGWSK1RJJBEUGSP0YGEE1IIS3EOZKKQY40FWTRZDLNB4' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4IWPTPZBLHK1ISW0CQEP0IBTOX3W4OSIRB1DHH3VWM3PYXWU
CLIENT_SECRET:QTGRBGWSK1RJJBEUGSP0YGEE1IIS3EOZKKQY40FWTRZDLNB4


In [67]:
request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180604',
    "section": "coffee",
    "near": "Ho Chi Minh",
    "radius": 1000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [68]:
print(data)

<Response [200]>


In [69]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [70]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Ho Chi Minh City', 'Ho Chi Minh City')

In [71]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 10.829369720714618, 'lng': 106.63348224158644},
  'sw': {'lat': 10.815630264993002, 'lng': 106.62468068103415}},
 7)

In [72]:
d["geocode"]

{'what': '',
 'where': 'ho chi minh',
 'center': {'lat': 10.82302, 'lng': 106.62965},
 'displayString': 'Ho Chi Minh City, Vietnam',
 'cc': 'VN',
 'geometry': {'bounds': {'ne': {'lat': 10.860618, 'lng': 106.744179},
   'sw': {'lat': 10.701005, 'lng': 106.599831}}},
 'slug': 'thanh-pho-ho-chi-minh',
 'longId': '72057594039494019'}

In [75]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [76]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [77]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 7


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4bf897585ec320a1ab1a89d3',
  'name': 'Bang Lang Tim Cafe',
  'location': {'lat': 10.824242092125864,
   'lng': 106.63073196637582,
   'labeledLatLngs': [{'label': 'display',
     'lat': 10.824242092125864,
     'lng': 106.63073196637582}],
   'cc': 'VN',
   'country': 'Việt Nam',
   'formattedAddress': ['Việt Nam']},
  'categories': [{'id': '4bf58dd8d48988d16d941735',
    'name': 'Café',
    'pluralName': 'Cafés',
    'shortName': 'Café',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-5-4bf897585ec320a1ab1a89d3-0'}

In [56]:
items[1]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4d4b7ccb61e0224bd61ba7e8',
  'name': '17 Cafe',
  'location': {'address': 'Tầng 17, Chung cư Phúc Yên, Phan Huy Ích, quận Gò Vấp',
   'crossStreet': 'Trường Chinh',
   'lat': 10.825452363902858,
   'lng': 106.62959395164766,
   'labeledLatLngs': [{'label': 'display',
     'lat': 10.825452363902858,
     'lng': 106.62959395164766}],
   'cc': 'VN',
   'city': 'Thành phố Hồ Chí Minh',
   'state': 'Thành phố Hồ Chí Minh',
   'country': 'Việt Nam',
   'formattedAddress': ['Tầng 17, Chung cư Phúc Yên, Phan Huy Ích, quận Gò Vấp (Trường Chinh)',
    'Thành phố Hồ Chí Minh',
    'Thành phố Hồ Chí Minh',
    'Việt Nam']},
  'categories': [{'id': '4bf58dd8d48988d16d941735',
    'name': 'Café',
    'pluralName': 'Cafés',
    'shortName': 'Café',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
     'suffix': '.p

In [107]:
df_raw = []
for item in items:
    venue = item['venue']
    name = venue['name']
    latitude = venue['location']['lat']
    longitude = venue['location']['lng']
    datarow = (name, latitude, longitude)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=['name','latitude','longitude'])

In [111]:
df.head(7)

,name,latitude,longitude
0,Bang Lang Tim Cafe,10.824242,106.630732
1,17 Cafe,10.825452,106.629594
2,New Day Cafe,10.824307,106.633082
3,Caffe D'Mang,10.828551,106.628840
4,Panda Cafe,10.828745,106.626817
5,Cafe Eden,10.828005,106.625081
6,Cafe Cây Tùng,10.816255,106.632443


In [112]:
hcm_center = d["geocode"]["center"]
hcm_center

{'lat': 10.82302, 'lng': 106.62965}

In [113]:
from folium import plugins


map_HCM = folium.Map(location=[10.82302, 106.62965], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["latitude"], row["longitude"]],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_HCM)

add_markers(df)
hm_data = df[["latitude", "longitude"]].to_numpy().tolist()
map_HCM.add_child(plugins.HeatMap(hm_data))

map_HCM

In [114]:
lat = 10.82302
lng = 106.62965
map_HoChiMinh = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Welcome to HCM City",
    color='green',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_HoChiMinh)
map_HoChiMinh